In [1]:
#! pip install semantic-kernel -U
#! pip install qdrant_client -U

In [2]:
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion, AzureTextEmbedding
from semantic_kernel.connectors.memory.qdrant import QdrantMemoryStore

In [3]:
kernel = sk.Kernel()

In [4]:
chat_service_id = "default"

embed_service_id="embedding"


oai_chat_service = AzureChatCompletion(
        service_id=chat_service_id,
        endpoint="",
        api_key="",
        deployment_name = "gpt-4o",
        api_version = "2024-08-01-preview",
        #kernel = kernel
)
 

embedding_gen = AzureTextEmbedding(
        service_id=embed_service_id,
        deployment_name = "text-embedding-ada-002", 
        endpoint = "",
        api_key="",
        api_version = "2023-05-15",
        #kernel = kernel
)
        
kernel.add_service(oai_chat_service)
kernel.add_service(embedding_gen)

In [5]:
from semantic_kernel.core_plugins.text_memory_plugin import TextMemoryPlugin
from semantic_kernel.memory.semantic_text_memory import SemanticTextMemory

In [6]:
qdrant_store = QdrantMemoryStore(vector_size=1536, url="http://localhost",port=6333)
await qdrant_store.create_collection('aboutMe')

In [18]:
memory = SemanticTextMemory(storage=qdrant_store, embeddings_generator=embedding_gen)
kernel.add_plugin(TextMemoryPlugin(memory), "TextMemoryPlugin")

KernelPlugin(name='TextMemoryPlugin', description=None, functions={'recall': KernelFunctionFromMethod(metadata=KernelFunctionMetadata(name='recall', plugin_name='TextMemoryPlugin', description='Recall a fact from the long term memory', parameters=[KernelParameterMetadata(name='ask', description='The information to retrieve', default_value=None, type_='str', is_required=True, type_object=<class 'str'>, schema_data={'type': 'string', 'description': 'The information to retrieve'}, include_in_function_choices=True), KernelParameterMetadata(name='collection', description='The collection to search for information.', default_value='generic', type_='str', is_required=False, type_object=<class 'str'>, schema_data={'type': 'string', 'description': 'The collection to search for information.'}, include_in_function_choices=True), KernelParameterMetadata(name='relevance', description='The relevance score, from 0.0 to 1.0; 1.0 means perfect match', default_value=0.75, type_='float', is_required=False

In [19]:
base_vectordb = 'aboutMe'

In [ ]:

async def populate_memory(memory: SemanticTextMemory) -> None:
    # Add some documents to the semantic memory
    await memory.save_information(base_vectordb, id="info1", text="Jack is Microsoft Cloud Advocate")
    await memory.save_information(base_vectordb, id="info2", text="Jack is ex-IBM employee")
    await memory.save_information(base_vectordb, id="info3", text="Jack is AI/ML Expert")
    await memory.save_information(base_vectordb, id="info4", text="OpenAI is a company that is developing artificial general intelligence (AGI) with widely distributed economic benefits.")


In [21]:
await populate_memory(memory)

In [22]:
ask = "who is Jack？"

memories = await memory.search(base_vectordb, ask, limit=2, min_relevance_score=0.77)

In [23]:
i = 0
for memory in memories:
    i = i + 1
    print(f"Top {i} Result: {memory.text} with score {memory.relevance}")

Top 1 Result: Jack is ex-IBM employee with score 0.85357594
Top 2 Result: Jack is AI/ML Expert with score 0.8478449
